# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [23]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 3

client_id = "FKiGMtBKKxeO3FyrSUPJ"
client_secret = "aCZo94oqqw"
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    # json 결과
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx * 10 + 1)
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
    #    response_body = response.read()
        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )
        news_data.append(result.json())
    #    print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [40]:
# print(news_data)
# print(len(news_data))
# print(news_data[0])
# print(news_data[0]['items'])
# print(len(news_data[0]['items']))
# print(news_data[0]['items'][2])
print(news_data[0]['items'][7]['link'])

http://www.irobotnews.com/news/articleView.html?idxno=25438


In [25]:
naver_news_link = []

for page in news_data:
    #print(page)
    page_news_link = []
    
    for item in page['items']:
        #print(item)
        temp_link = item['link']
        #print(temp_link)
        if "naver" in temp_link:
            page_news_link.append(temp_link)
    
    naver_news_link.append(page_news_link)
        

# 사이트 확인하기에 편한 코드 구조.
for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=417&aid=0000710301
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002106033
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=018&aid=0004971460
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=015&aid=0004570130
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=138&aid=0002105960
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004928707
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=417&aid=0000709408


In [9]:
!pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [26]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm.notebook import tqdm
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [27]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome('driver/chromedriver')

In [28]:
naver_news_title = []
naver_news_content = []


for n in tqdm(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=417&aid=0000710301



https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=138&aid=0002106033
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=018&aid=0004971460



https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=015&aid=0004570130
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=138&aid=0002105960
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004928707
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=417&aid=0000709408


['에듀테크 스타트업 ‘지니로봇’ 프리 시리즈A 투자 유치']
['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    IBK 기업은행의 창업육성 프로그램  IBK 창공(創工) 구로 5기 육성기업으로  TIPS  운영사인 액셀러레이터 씨엔티테크㈜(대표 전화성)가 함께 육성을 맡고 있는\xa0에듀테크 스타트업 ㈜지니로봇(대표 이은승)이 최근  IBK 기업은행과 신용보증기금으로부터 7억원 규모의 ‘프리 시리즈 A’  투자유치를 완료했다고 밝혔다. 지니로봇은 비대면 양방향 다자간 화상 교육 플랫폼 ‘ LIMS  ( Learning   Interactive   Multiple   Service )’와 코딩,  STEAM  및 인공지능 교육이 가능한 올인원 교육용 로봇 ‘지니봇’을 개발, 제조 및 판매하고 있는 에듀테크 스타트업이다.\xa0 지니로봇이   IBK 기업은행과 신용보증기금으로부터 7억원 규모의 ‘프리 시리즈 A’  투자유치를 완료했다고 밝혔다. (씨엔티테크 제공) 지니봇은 언플러그드 카드 코딩부터 전용 앱, 엔트리, 스크래치  3.0 , 파이썬 등 활용이 가능하며, 비대면 환경에서  LIMS 를 통해 선생님과 학생이 양

In [29]:
print(naver_news_title[0])

['에듀테크 스타트업 ‘지니로봇’ 프리 시리즈A 투자 유치']


In [30]:
print(naver_news_content[0])

['   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    IBK 기업은행의 창업육성 프로그램  IBK 창공(創工) 구로 5기 육성기업으로  TIPS  운영사인 액셀러레이터 씨엔티테크㈜(대표 전화성)가 함께 육성을 맡고 있는\xa0에듀테크 스타트업 ㈜지니로봇(대표 이은승)이 최근  IBK 기업은행과 신용보증기금으로부터 7억원 규모의 ‘프리 시리즈 A’  투자유치를 완료했다고 밝혔다. 지니로봇은 비대면 양방향 다자간 화상 교육 플랫폼 ‘ LIMS  ( Learning   Interactive   Multiple   Service )’와 코딩,  STEAM  및 인공지능 교육이 가능한 올인원 교육용 로봇 ‘지니봇’을 개발, 제조 및 판매하고 있는 에듀테크 스타트업이다.\xa0 지니로봇이   IBK 기업은행과 신용보증기금으로부터 7억원 규모의 ‘프리 시리즈 A’  투자유치를 완료했다고 밝혔다. (씨엔티테크 제공) 지니봇은 언플러그드 카드 코딩부터 전용 앱, 엔트리, 스크래치  3.0 , 파이썬 등 활용이 가능하며, 비대면 환경에서  LIMS 를 통해 선생님과 학생이 양방향으로 평가, 분석 및 맞춤형 교육이 가능하도록 개발했다.\xa0 최근 ‘클라우드 플랫폼에 연결된 피어 단말기들을 이용하여 코딩 교육을 제공하는 시스템 및 방법’ 특허 등록이 완료되어 기존 등록된 5개의 특허와 함께 보다 안정적으로 사업을 할 수 있는 발판을 마련했다. 이은승 지니로봇 대표는 “코로나  19 로 인해 비대면 수업이 시작되면서 학습 격차가 점점 벌어지면서 학습의 양극화 현상이 현실화되고 있다”며 학생들이 스스로 공부할 수 있는 환경이 갖춰져야 하는데 “ LIMS ”는 개인 맞춤형 학습 진단·관리를 통해 학생 개개인에게 맞춤형 학습 교육을 제공할 것”이라고 전했다. 지니로봇은 창업진흥원 예비창업패키지 지원사업 1억원을 시작으로 중소벤처기업부  TIPS , 신용

In [31]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

1
1


In [32]:
with open("data/naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)
    
with open("data/naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)